In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from UnarySim.sw.kernel.linear import UnaryLinear
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError


In [17]:
in_feature = 1
out_feature = 2
mode = "unipolar"
scaled = False
rng = "Sobol"
bias=True

fc = torch.nn.Linear(in_feature, out_feature, bias=bias)
if mode is "unipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(256).round().div(256)
elif mode is "bipolar":
    fc.weight.data = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(256).round().div(256)
    if bias is True:
        fc.bias.data = torch.rand(1, out_feature).mul(2).sub(1).mul(256).round().div(256)
ufc = UnaryLinear(in_feature, out_feature, 256, fc.weight, fc.bias, mode=mode, scaled=scaled, bias=bias)

print("weight:")
print(fc.weight.data)
print(fc.weight.data.t())
print(fc.weight.size())
print(fc.weight.data.t().size())

if bias is True:
    print("\nbias:")
    print(fc.bias)
    print(fc.bias.size())

iVec = (torch.rand(1, in_feature)*256).round()/256
oVec = fc(iVec)

print("\ninput:")
print(iVec)
print("\noutput:")
print(oVec)

if scaled is False:
    oVec = oVec.clamp(-1.,1.).mul(256).floor()/256
else:
    if bias is True:
        oVec = oVec.div(in_feature+1).mul(256).floor()/256
    else:
        oVec = oVec.div(in_feature).mul(256).floor()/256

print("\nquantized output:")
print(oVec)

Linear(in_features=1, out_features=2, bias=True)
self.buf_wght Parameter containing:
tensor([[239],
        [149]])
self.buf_wght_bs BSGen()
rng_wght_idx: tensor([[0],
        [0]])
weight:
tensor([[0.9336],
        [0.5820]])
tensor([[0.9336, 0.5820]])
torch.Size([2, 1])
torch.Size([1, 2])

bias:
Parameter containing:
tensor([[0.9688, 0.5938]], requires_grad=True)
torch.Size([1, 2])

input:
tensor([[0.1328]])

output:
tensor([[1.0927, 0.6711]], grad_fn=<AddmmBackward>)

quantized output:
tensor([[1.0000, 0.6680]], grad_fn=<DivBackward0>)


In [16]:
iVecSource = SourceGen(iVec, bitwidth=8, mode=mode).Gen()

iVecRNG = RNG(8, 1, rng).Out()
iVecBS = BSGen(iVecSource, iVecRNG)

iVecPP = ProgressiveError(iVec, mode=mode)
oVecPP = ProgressiveError(oVec, mode=mode)

with torch.no_grad():
    idx = torch.zeros(iVecSource.size()).type(torch.long)
    for i in range(256):
        iBS = iVecBS.Gen(idx + i)
        iVecPP.Monitor(iBS)
        oVecU = ufc(iBS)

        oVecPP.Monitor(oVecU)
    print("input error: ", min(min(iVecPP.Report())), max(max(iVecPP.Report())))
    print("output error:", min(min(oVecPP.Report())), max(max(oVecPP.Report())))
    print(oVecPP.Report())
    print(oVecPP.one_cnt)
    print(oVecPP.out_pp)



input error:  tensor(0.) tensor(0.)
output error: tensor(0.1055) tensor(0.1055)
tensor([[0.1055]])
tensor([[55.]])
tensor([[0.2148]])
